<h2>Baseline Code</h2>

In [1]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model, to_categorical
import warnings
import re

In [2]:
ls "."

 C 드라이브의 볼륨: SSD
 볼륨 일련 번호: BCA4-98AF

 C:\Users\inhoinno\DACON\2020_DACON-Kaggle\소설저자AI 디렉터리

2020-11-26  오후 07:29    <DIR>          .
2020-11-26  오후 07:29    <DIR>          ..
2020-11-25  오전 12:07    <DIR>          .ipynb_checkpoints
2020-11-24  오전 10:49            33,874 Augmentation.ipynb
2020-11-24  오전 11:15    <DIR>          dataset
2020-11-05  오후 05:31    <DIR>          ensemble
2020-10-09  오후 05:05         1,327,724 Logistic_regression.ipynb
2020-10-29  오후 06:12         3,959,848 NPL_base_5layer.h5
2020-11-12  오전 11:23            35,818 NPL_BaseLine.ipynb
2020-11-21  오후 11:47           992,363 NPL_BaseLine-CV.ipynb
2020-11-26  오후 07:29            40,803 NPL_BaseLine-CV-Copy1.ipynb
2020-11-05  오전 12:07           934,892 NPL_BaseLine-LSTM.ipynb
2020-11-12  오전 11:23           942,567 NPL_BaseLine-ONE-HOT.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV.ipynb
2020-11-20  오후 05:49           115,342 NPL_Base-Tfidf-CV-Copy1.ipynb
2020-11-12  오전 11:23           927,110 NP

In [3]:
#파일 불러오기
train = pd.read_csv('open/train_processed_500.csv', encoding = 'utf-8')
test = pd.read_csv('open/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('open/sample_submission.csv', encoding = 'utf-8')

In [4]:
train

,index,text,author,length
0,0,He was almost choking. There was so much so mu...,3,235
1,1,Your sister asked for it I suppose,2,34
2,2,She was engaged one day as she walked in peru...,1,312
3,3,The captain was in the porch keeping himself c...,4,305
4,4,Have mercy gentlemen odin flung up his hands. ...,3,215
...,...,...,...,...
183463,183463,To be urging her opinion against Sir odins wa...,1,571
183464,183464,After many tears an unwritten contract was dr...,3,536
183465,183465,They express the consciousness that you have ...,3,525
183466,183466,And of course he knows himself that he is doi...,3,515


In [5]:
train.author.describe()

count    183468.000000
mean          2.049088
std           1.285455
min           0.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           4.000000
Name: author, dtype: float64

<h2>전처리

In [6]:
#부호를 제거해주는 함수

def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [7]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author,length
0,0,He was almost choking There was so much so muc...,3,235
1,1,Your sister asked for it I suppose,2,34
2,2,She was engaged one day as she walked in peru...,1,312
3,3,The captain was in the porch keeping himself c...,4,305
4,4,Have mercy gentlemen odin flung up his hands D...,3,215
...,...,...,...,...
183463,183463,To be urging her opinion against Sir odins wa...,1,571
183464,183464,After many tears an unwritten contract was dr...,3,536
183465,183465,They express the consciousness that you have ...,3,525
183466,183466,And of course he knows himself that he is doi...,3,515


1. 불용어 제거 remove_stopwords

In [8]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [9]:
idx_under_20 = train[train['length'] < 20].index
train = train.drop(idx_under_20)
train.describe()

,index,author,length
count,181013.000000,181013.000000,181013.000000
mean,91514.586997,2.049146,164.537862
std,53025.418222,1.286300,178.695183
min,0.000000,0.000000,20.000000
25%,45490.000000,1.000000,65.000000
50%,91441.000000,2.000000,113.000000
75%,137458.000000,3.000000,189.000000
max,183467.000000,4.000000,2454.000000


In [10]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num)#.apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num)#.apply(remove_stopwords)

In [11]:
idx_under_20 = train[train['length'] < 20].index
train = train.drop(idx_under_20)
train = train.reset_index()
train.describe()

,level_0,index,author,length
count,181013.000000,181013.000000,181013.000000,181013.000000
mean,91514.586997,91514.586997,2.049146,164.537862
std,53025.418222,53025.418222,1.286300,178.695183
min,0.000000,0.000000,0.000000,20.000000
25%,45490.000000,45490.000000,1.000000,65.000000
50%,91441.000000,91441.000000,2.000000,113.000000
75%,137458.000000,137458.000000,3.000000,189.000000
max,183467.000000,183467.000000,4.000000,2454.000000


In [12]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [13]:
train
#문체를 다 짤라먹었는데 왜 다 짤라먹었을까????
# , 활용 "" 활용같은것도 중요할것 같은데

,level_0,index,text,author,length
0,0,0,he was almost choking there was so much so muc...,3,235
1,1,1,your sister asked for it i suppose,2,34
2,2,2,she was engaged one day as she walked in peru...,1,312
3,3,3,the captain was in the porch keeping himself c...,4,305
4,4,4,have mercy gentlemen odin flung up his hands d...,3,215
...,...,...,...,...,...
181008,183463,183463,to be urging her opinion against sir odins wa...,1,571
181009,183464,183464,after many tears an unwritten contract was dr...,3,536
181010,183465,183465,they express the consciousness that you have ...,3,525
181011,183466,183466,and of course he knows himself that he is doi...,3,515


<h2> Stratified K-Fold Cross Validation

In [14]:
n_fold = 5
n_class = 5
seed = 777


In [15]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [16]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np 

In [17]:
maxlen = 1000 #100개 이후 단어는 버려
max_words = 20000 #데이터셋에서 가장 빈도 높은 1만개의 단어만 사용
embedding_dim=32

In [18]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['text'])
sequences = tokenizer.texts_to_sequences(train['text'])

In [19]:
word_index = tokenizer.word_index
print('%s개의 고유한 토큰 확인'%len(word_index))

47272개의 고유한 토큰 확인


In [20]:
data = pad_sequences(sequences, maxlen= maxlen)
print('데이터 텐서 크기',data.shape)

데이터 텐서 크기 (181013, 1000)


In [21]:
y_train = train['author']

<h2> Keras 모델링

In [22]:
# train test 분리
X_test = np.array([x for x in test['text']])

In [23]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, maxlen=maxlen)

In [24]:
#임베딩 한번 안해볼게

<h3> 가벼운 NLP모델 생성

In [25]:
"""#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])"""

"#가벼운 NLP모델 생성\nmodel = tf.keras.Sequential([\n    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),\n    tf.keras.layers.GlobalAveragePooling1D(),\n    tf.keras.layers.Dense(24, activation='relu'),\n    tf.keras.layers.Dense(5, activation='softmax')\n])"

In [26]:
def get_base_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    
    return model

<h3> Stratified KFold NLP 모델 학습

In [27]:
trn = data
y = y_train
tst = test_padded
(trn.shape,y.shape,tst.shape)

((181013, 1000), (181013,), (19617, 1000))

In [28]:
tst

array([[    0,     0,     0, ...,     6,    23,  1328],
       [    0,     0,     0, ...,    79,    43, 12394],
       [    0,     0,     0, ...,    60,     8,    38],
       ...,
       [    0,     0,     0, ...,     7,    18,   145],
       [    0,     0,     0, ...,    28,    20,  3832],
       [    0,     0,     0, ...,     3,   688,  1865]])

In [29]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))

for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    
    clf = get_base_model()
    
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True)

    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                            patience=3, min_lr=1e-6, mode='min', verbose=1)

    clf.fit(trn[i_trn], 
            y[i_trn],
            validation_data=(trn[i_val], (y[i_val])),
            epochs=100,
            #batch_size = 64,
            callbacks=[es, rlr])
    p_val[i_val, :] = clf.predict(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
Epoch 1/100


InternalError: 2 root error(s) found.
  (0) Internal:  Blas GEMM launch failed : a.shape=(32, 32), b.shape=(32, 24), m=32, n=24, k=32
	 [[node sequential/dense/MatMul (defined at <ipython-input-29-88850485ff7e>:15) ]]
	 [[gradient_tape/sequential/embedding/embedding_lookup/Reshape/_38]]
  (1) Internal:  Blas GEMM launch failed : a.shape=(32, 32), b.shape=(32, 24), m=32, n=24, k=32
	 [[node sequential/dense/MatMul (defined at <ipython-input-29-88850485ff7e>:15) ]]
0 successful operations.
0 derived errors ignored. [Op:__inference_train_function_786]

Function call stack:
train_function -> train_function


<h3> Submission

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = p_tst
sample_submission

In [ ]:
sample_submission.to_csv('open/sub/submission_BASELINE_CV_processed4.csv', index = False, encoding = 'utf-8')
